In [143]:
import pandas as pd
import numpy as np
import re as re

In [144]:
Exportar = 'J:/My Drive/Forraje/Exportar.xls'
df_Exportar = pd.read_excel(Exportar)

In [145]:
df_Exportar = df_Exportar.rename(columns={'Descripción': 'Descripcion'})
df_Exportar = df_Exportar.rename(columns={'Costo Unitário': 'Costo'})

In [146]:
Marcas = 'J:/My Drive/Forraje/Marcas y subcategorías.xlsx'
df_Marcas = pd.read_excel(Marcas)

In [147]:
df = pd.merge(df_Exportar, df_Marcas, how='left', on='Descripcion')

In [148]:
df = df[['Descripcion', 'Categoría_y', 'Subcategoria', 'Marca_y', 'Costo', 'Precio', 'Markup %']]

In [149]:
df = df.rename(columns={'Categoría_y': 'Categoria'})
df = df.rename(columns={'Marca_y': 'Marca'})
df = df.rename(columns={'Markup %': 'Markup'})

In [150]:
df['Markup'] = (
    df['Markup']
    .str.replace(',', '.', regex=False) 
    .str.replace('%', '', regex=False) 
    .astype(float) 
)

In [151]:
df.columns

Index(['Descripcion', 'Categoria', 'Subcategoria', 'Marca', 'Costo', 'Precio',
       'Markup'],
      dtype='object')

In [152]:
Aumentos_Subcategoria = {
    "Purina": 5
}

In [153]:
Aumentos_Marcas = {
    "9 Lives": 8,
    "Zimpi": 6,
    "Compinche": 6,
    "Pedigree": 8.5,
    "Whiskas": 8.5,
    "Eukanuba": 9,
    "Royal Canin": 9
}

In [154]:
Aumentos_Nombres = {
    "Dentastix": 20,
    "Pedigree/Whiskas Latas": 20,
    "Pedigree/Whiskas Pouch": 20
}

In [155]:
df_Resultados = pd.DataFrame()

In [161]:
df[df['Descripcion'].str.contains('Dentastix', case=False, na=False)]

,Descripcion,Categoria,Subcategoria,Marca,Costo,Precio,Markup
1410,Pedigree Dentastix Razas Medianas,Mascotas,Pedigree,Pedigree,328.0,500.0,52.4
1411,Pedigree Dentastix Razas Medianas x 3 u.,Mascotas,Pedigree,Pedigree,943.0,1500.0,59.0
1412,Pedigree Dentastix Razas Medianas x 7 u.,Mascotas,Pedigree,Pedigree,2462.0,3500.0,42.1
1413,Pedigree Dentastix Razas Pequeñas,Mascotas,Pedigree,Pedigree,270.0,500.0,85.1
1414,Pedigree Dentastix Razas Pequeñas x 3 u.,Mascotas,Pedigree,Pedigree,800.0,1200.0,50.0
1415,Pedigree Dentastix Razas Pequeñas x 7 u.,Mascotas,Pedigree,Pedigree,1553.0,2400.0,54.5


In [157]:
for Subcategoria, Aumento in Aumentos_Subcategoria.items():
    df_Provisorio = df[df['Subcategoria'] == Subcategoria]
    df_Provisorio = df_Provisorio[['Descripcion', 'Precio', 'Costo', 'Markup']]
    
    # Convertir las columnas 'Costo' y 'Precio' a valores numéricos.
    df_Provisorio['Costo'] = pd.to_numeric(df_Provisorio['Costo'], errors='coerce')
    df_Provisorio['Precio'] = pd.to_numeric(df_Provisorio['Precio'], errors='coerce')
    
    # Aplicar el aumento al costo.
    df_Provisorio['Costo'] = df_Provisorio['Costo'] * (1 + (Aumento / 100))
    
    # Calcular el nuevo precio y redondearlo al múltiplo.
    df_Provisorio['Precio'] = df_Provisorio['Costo'] * (1 + (df_Provisorio['Markup'] / 100))
    df_Provisorio['Precio'] = (df_Provisorio['Precio'] / 50).round() * 50 
    
    # Concatenar los resultados.
    df_Resultados = pd.concat([df_Resultados, df_Provisorio])


In [158]:
for Marca, Aumento in Aumentos_Marcas.items():
    df_Provisorio = df[df['Marca'] == Marca]
    df_Provisorio = df_Provisorio[['Descripcion', 'Precio', 'Costo', 'Markup']]
    
    # Convertir las columnas 'Costo' y 'Precio' a valores numéricos.
    df_Provisorio['Costo'] = pd.to_numeric(df_Provisorio['Costo'], errors='coerce')
    df_Provisorio['Precio'] = pd.to_numeric(df_Provisorio['Precio'], errors='coerce')
    
    # Aplicar el aumento al costo.
    df_Provisorio['Costo'] = df_Provisorio['Costo'] * (1 + (Aumento / 100))
    
    # Calcular el nuevo precio y redondearlo al múltiplo.
    df_Provisorio['Precio'] = df_Provisorio['Costo'] * (1 + (df_Provisorio['Markup'] / 100))
    df_Provisorio['Precio'] = (df_Provisorio['Precio'] / 50).round() * 50 
    
    # Concatenar los resultados.
    df_Resultados = pd.concat([df_Resultados, df_Provisorio])

In [159]:
for Nombre, Aumento in Aumentos_Nombres.items():
    df_Provisorio = df[df['Descripcion'].str.contains(Nombre, case=False, na=False)]
    df_Provisorio = df_Provisorio[['Descripcion', 'Precio', 'Costo', 'Markup']]
    
    # Convertir las columnas 'Costo' y 'Precio' a valores numéricos.
    df_Provisorio['Costo'] = pd.to_numeric(df_Provisorio['Costo'], errors='coerce')
    df_Provisorio['Precio'] = pd.to_numeric(df_Provisorio['Precio'], errors='coerce')
    
    # Aplicar el aumento al costo.
    df_Provisorio['Costo'] = df_Provisorio['Costo'] * (1 + (Aumento / 100))
    
    # Calcular el nuevo precio y redondearlo al múltiplo.
    df_Provisorio['Precio'] = df_Provisorio['Costo'] * (1 + (df_Provisorio['Markup'] / 100))
    df_Provisorio['Precio'] = (df_Provisorio['Precio'] / 50).round() * 50 
    
    # Concatenar los resultados.
    df_Resultados = pd.concat([df_Resultados, df_Provisorio])

In [160]:
df_Resultados

,Descripcion,Precio,Costo,Markup
390,Cat Chow Carne/Pescado,5050.0,3339.00,50.9
391,Cat Chow Carne/Pescado x 15 kg.,66100.0,50091.30,32.0
392,Cat Chow Carne/Pescado x 8 kg.,38950.0,29466.15,32.2
393,Cat Chow Esterilizados,5250.0,3582.60,46.2
394,Cat Chow Esterilizados x 15 kg.,80000.0,60547.20,32.1
...,...,...,...,...
1411,Pedigree Dentastix Razas Medianas x 3 u.,1800.0,1131.60,59.0
1412,Pedigree Dentastix Razas Medianas x 7 u.,4200.0,2954.40,42.1
1413,Pedigree Dentastix Razas Pequeñas,600.0,324.00,85.1
1414,Pedigree Dentastix Razas Pequeñas x 3 u.,1450.0,960.00,50.0
